In [2]:
import pandas as pd

# Load the dataset
file_path = r'C:\Users\KIIT\Dropbox\Documents\Internship\ReBit\Daily Reports\Implementing_Methods\Main-aggregates-national-accounts-(International-Data-Cooperation).csv'
data = pd.read_csv(file_path)

# Display the first few rows of the dataset to understand its structure
print(data.head())

                                           KEY FREQ ADJUSTMENT REF_AREA  \
0  IDCM.A.N.6O.W0.S1.S1.B.B5G._Z._Z._Z.USD.V.N    A          N       6O   
1  IDCM.A.N.6O.W0.S1.S1.B.B5G._Z._Z._Z.USD.V.N    A          N       6O   
2  IDCM.A.N.6O.W0.S1.S1.B.B5G._Z._Z._Z.USD.V.N    A          N       6O   
3  IDCM.A.N.6O.W0.S1.S1.B.B5G._Z._Z._Z.USD.V.N    A          N       6O   
4  IDCM.A.N.6O.W0.S1.S1.B.B5G._Z._Z._Z.USD.V.N    A          N       6O   

  COUNTERPART_AREA REF_SECTOR COUNTERPART_SECTOR ACCOUNTING_ENTRY  STO  \
0               W0         S1                 S1                B  B5G   
1               W0         S1                 S1                B  B5G   
2               W0         S1                 S1                B  B5G   
3               W0         S1                 S1                B  B5G   
4               W0         S1                 S1                B  B5G   

  INSTR_ASSET  ... REF_PERIOD_DETAIL REF_YEAR_PRICE REPYEAREND REPYEARSTART  \
0          _Z  ...       

In [3]:
import gym
from gym import spaces
import numpy as np
import pandas as pd

class CentralBankingEnv(gym.Env):
    def __init__(self, data, target_inflation):
        super(CentralBankingEnv, self).__init__()
        self.data = data
        self.target_inflation = target_inflation
        self.current_step = 0
        
        # Define observation space based on selected columns (excluding non-numeric and irrelevant columns)
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(1,), dtype=np.float32)
        self.action_space = spaces.Discrete(3)  # Example: 0 = decrease rate, 1 = maintain, 2 = increase rate

    def step(self, action):
        # Take an action and move to the next step
        reward = self.calculate_reward(action)
        self.current_step += 1
        if self.current_step >= len(self.data):
            done = True
            next_state = np.array([np.nan])
        else:
            next_state = np.array([self.data.iloc[self.current_step]['OBS_VALUE']])
            done = np.isnan(next_state).any()  # Check for NaN values
        return next_state, reward, done, {}

    def reset(self):
        self.current_step = 0
        return np.array([self.data.iloc[self.current_step]['OBS_VALUE']])

    def calculate_reward(self, action):
        current_inflation = self.data.iloc[self.current_step]['OBS_VALUE']
        
        # Adjust inflation based on the action taken
        if action == 0:  # decrease rate
            adjusted_inflation = current_inflation - 0.1
        elif action == 1:  # maintain rate
            adjusted_inflation = current_inflation
        else:  # increase rate
            adjusted_inflation = current_inflation + 0.1
        
        # Reward is the negative absolute deviation from the target inflation
        reward = -abs(adjusted_inflation - self.target_inflation)
        return reward

    def render(self, mode='human', close=False):
        # Optionally implement rendering
        print(f'Step: {self.current_step}, Inflation: {self.data.iloc[self.current_step]["OBS_VALUE"]}')


# Load the dataset
file_path = r'C:\Users\KIIT\Dropbox\Documents\Internship\ReBit\Daily Reports\Implementing_Methods\Main-aggregates-national-accounts-(International-Data-Cooperation).csv'
data = pd.read_csv(file_path)

# Initialize the environment with a target inflation rate
target_inflation = 2.0
env = CentralBankingEnv(data, target_inflation)

# Reset the environment
state = env.reset()

# Example run through the environment with random actions
done = False
while not done:
    action = env.action_space.sample()  # Random action
    next_state, reward, done, _ = env.step(action)
    env.render()


Step: 1, Inflation: 30272258.0
Step: 2, Inflation: 31377072.0
Step: 3, Inflation: 32532831.0
Step: 4, Inflation: 34578437.0
Step: 5, Inflation: 36527206.0
Step: 6, Inflation: 39350893.0
Step: 7, Inflation: 41307114.0
Step: 8, Inflation: 42513604.0
Step: 9, Inflation: 42580764.0
Step: 10, Inflation: 44788795.0
Step: 11, Inflation: 46995374.0
Step: 12, Inflation: 48661455.0
Step: 13, Inflation: 50613928.0
Step: 14, Inflation: 52532769.0
Step: 15, Inflation: 54257222.0
Step: 16, Inflation: 56345441.0
Step: 17, Inflation: 58920912.0
Step: 18, Inflation: 24657516.0
Step: 19, Inflation: 25548166.0
Step: 20, Inflation: 26453214.0
Step: 21, Inflation: 27446195.0
Step: 22, Inflation: 29217691.0
Step: 23, Inflation: 30863615.0
Step: 24, Inflation: 33237795.0
Step: 25, Inflation: 34816576.0
Step: 26, Inflation: 35639084.0
Step: 27, Inflation: 34608857.0
Step: 28, Inflation: 36621786.0
Step: 29, Inflation: 38497172.0
Step: 30, Inflation: 39883870.0
Step: 31, Inflation: 41456635.0
Step: 32, Inflati

In [4]:
# Q-Learning Algorithm (Value Based Model)

In [5]:
import numpy as np

# Discretize the state space
state_bins = 10  # Number of bins to discretize the state space
state_space = np.linspace(min(data['OBS_VALUE']), max(data['OBS_VALUE']), state_bins)

def discretize_state(state):
    return np.digitize(state, state_space) - 1

# Initialize Q-table
q_table = np.zeros((state_bins, env.action_space.n))

# Hyperparameters
alpha = 0.1  # Learning rate
gamma = 0.99  # Discount factor
epsilon = 0.1  # Exploration rate

# Training loop
for episode in range(1000):
    state = discretize_state(env.reset())
    done = False
    steps = 0
    while not done:
        if np.random.rand() < epsilon:
            action = env.action_space.sample()  # Explore action space
        else:
            action = np.argmax(q_table[state])  # Exploit learned values
        next_state, reward, done, _ = env.step(action)
        if np.isnan(next_state).any():
            break  # Stop if next state is NaN
        next_state = discretize_state(next_state)
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        new_value = old_value + alpha * (reward + gamma * next_max - old_value)
        q_table[state, action] = new_value
        state = next_state
        steps += 1
        if steps > 1000:  # Prevent infinite loops
            done = True
    print(f"Episode {episode} completed in {steps} steps.")


Episode 0 completed in 999 steps.
Episode 1 completed in 999 steps.
Episode 2 completed in 999 steps.
Episode 3 completed in 999 steps.
Episode 4 completed in 999 steps.
Episode 5 completed in 999 steps.
Episode 6 completed in 999 steps.
Episode 7 completed in 999 steps.
Episode 8 completed in 999 steps.
Episode 9 completed in 999 steps.
Episode 10 completed in 999 steps.
Episode 11 completed in 999 steps.
Episode 12 completed in 999 steps.
Episode 13 completed in 999 steps.
Episode 14 completed in 999 steps.
Episode 15 completed in 999 steps.
Episode 16 completed in 999 steps.
Episode 17 completed in 999 steps.
Episode 18 completed in 999 steps.
Episode 19 completed in 999 steps.
Episode 20 completed in 999 steps.
Episode 21 completed in 999 steps.
Episode 22 completed in 999 steps.
Episode 23 completed in 999 steps.
Episode 24 completed in 999 steps.
Episode 25 completed in 999 steps.
Episode 26 completed in 999 steps.
Episode 27 completed in 999 steps.
Episode 28 completed in 999 st

In [6]:
# SARSA (Value Based Model)

In [7]:
def sarsa(env, num_episodes=1000, alpha=0.1, gamma=0.99, epsilon=0.1):
    state_bins = 10  # Number of bins to discretize the state space
    state_space = np.linspace(min(env.data['OBS_VALUE']), max(env.data['OBS_VALUE']), state_bins)

    def discretize_state(state):
        return np.digitize(state, state_space) - 1

    # Initialize Q-table
    q_table = np.zeros((state_bins, env.action_space.n))

    # Training loop
    for episode in range(num_episodes):
        state = discretize_state(env.reset())
        action = epsilon_greedy_policy(q_table, state, epsilon)
        done = False
        steps = 0
        
        while not done:
            next_state, reward, done, _ = env.step(action)
            if np.isnan(next_state).any():
                break  # Stop if next state is NaN
            next_state = discretize_state(next_state)
            next_action = epsilon_greedy_policy(q_table, next_state, epsilon)
            
            # SARSA update
            old_value = q_table[state, action]
            next_value = q_table[next_state, next_action]
            new_value = old_value + alpha * (reward + gamma * next_value - old_value)
            q_table[state, action] = new_value
            
            state = next_state
            action = next_action
            steps += 1
            if steps > 1000:  # Prevent infinite loops
                done = True
        
        print(f"Episode {episode} completed in {steps} steps.")

    return q_table

def epsilon_greedy_policy(q_table, state, epsilon):
    if np.random.rand() < epsilon:
        return np.random.choice(env.action_space.n)  # Explore action space
    else:
        return np.argmax(q_table[state])  # Exploit learned values

q_table = sarsa(env, num_episodes=1000, alpha=0.1, gamma=0.99, epsilon=0.1)

Episode 0 completed in 999 steps.
Episode 1 completed in 999 steps.
Episode 2 completed in 999 steps.
Episode 3 completed in 999 steps.
Episode 4 completed in 999 steps.
Episode 5 completed in 999 steps.
Episode 6 completed in 999 steps.
Episode 7 completed in 999 steps.
Episode 8 completed in 999 steps.
Episode 9 completed in 999 steps.
Episode 10 completed in 999 steps.
Episode 11 completed in 999 steps.
Episode 12 completed in 999 steps.
Episode 13 completed in 999 steps.
Episode 14 completed in 999 steps.
Episode 15 completed in 999 steps.
Episode 16 completed in 999 steps.
Episode 17 completed in 999 steps.
Episode 18 completed in 999 steps.
Episode 19 completed in 999 steps.
Episode 20 completed in 999 steps.
Episode 21 completed in 999 steps.
Episode 22 completed in 999 steps.
Episode 23 completed in 999 steps.
Episode 24 completed in 999 steps.
Episode 25 completed in 999 steps.
Episode 26 completed in 999 steps.
Episode 27 completed in 999 steps.
Episode 28 completed in 999 st

In [8]:
# REINFORCE (Policy Based Method)

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
import gym
from gym import spaces
import numpy as np
import pandas as pd

class CentralBankingEnv(gym.Env):
    def __init__(self, data, target_inflation):
        super(CentralBankingEnv, self).__init__()
        self.data = data
        self.target_inflation = target_inflation
        self.current_step = 0

        # Define observation space based on selected columns (excluding non-numeric and irrelevant columns)
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(1,), dtype=np.float32)
        self.action_space = spaces.Discrete(3)  # Example: 0 = decrease rate, 1 = maintain, 2 = increase rate

    def step(self, action):
        reward = self.calculate_reward(action)
        self.current_step += 1
        if self.current_step >= len(self.data):
            done = True
            next_state = np.array([np.nan])
        else:
            next_state = np.array([self.data.iloc[self.current_step]['OBS_VALUE']])
            done = np.isnan(next_state).any()  # Check for NaN values
        return next_state, reward, done, {}

    def reset(self):
        self.current_step = 0
        return np.array([self.data.iloc[self.current_step]['OBS_VALUE']])

    def calculate_reward(self, action):
        current_inflation = self.data.iloc[self.current_step]['OBS_VALUE']
        if action == 0:  # decrease rate
            adjusted_inflation = current_inflation - 0.1
        elif action == 1:  # maintain rate
            adjusted_inflation = current_inflation
        else:  # increase rate
            adjusted_inflation = current_inflation + 0.1
        reward = -abs(adjusted_inflation - self.target_inflation)
        return reward

    def render(self, mode='human', close=False):
        print(f'Step: {self.current_step}, Inflation: {self.data.iloc[self.current_step]["OBS_VALUE"]}')

# Load the dataset
file_path = r'C:\Users\KIIT\Dropbox\Documents\Internship\ReBit\Daily Reports\Implementing_Methods\Main-aggregates-national-accounts-(International-Data-Cooperation).csv'
data = pd.read_csv(file_path)

# Initialize the environment with a target inflation rate
target_inflation = 2.0
env = CentralBankingEnv(data, target_inflation)

class PolicyNetwork(nn.Module):
    def __init__(self, input_size, output_size):
        super(PolicyNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, output_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.softmax(self.fc2(x), dim=-1)
        return x

policy_net = PolicyNetwork(env.observation_space.shape[0], env.action_space.n)
optimizer = optim.Adam(policy_net.parameters(), lr=0.01)
gamma = 0.99

def select_action(state):
    state = torch.from_numpy(state).float().unsqueeze(0)
    probs = policy_net(state)
    m = torch.distributions.Categorical(probs)
    action = m.sample()
    return action.item(), m.log_prob(action)

for episode in range(1000):
    state = env.reset()
    rewards = []
    log_probs = []
    print(f"Episode {episode + 1} started.")
    
    for t in range(1000):
        action, log_prob = select_action(state)
        next_state, reward, done, _ = env.step(action)
        rewards.append(reward)
        log_probs.append(log_prob)
        state = next_state
        if done:
            break
    print(f"Episode {episode + 1} finished after {t + 1} steps.")
    
    R = 0
    policy_loss = []
    returns = []
    for r in rewards[::-1]:
        R = r + gamma * R
        returns.insert(0, R)

    returns = torch.tensor(returns)
    returns = (returns - returns.mean()) / (returns.std() + 1e-9)

    for log_prob, R in zip(log_probs, returns):
        policy_loss.append(-log_prob * R)

    optimizer.zero_grad()
    policy_loss = torch.cat(policy_loss).sum()
    policy_loss.backward()
    optimizer.step()
    
    print(f"Episode {episode + 1} policy loss: {policy_loss.item()}")
    print(f"Episode {episode + 1} total reward: {sum(rewards)}")

print("Training completed.")


Episode 1 started.
Episode 1 finished after 1000 steps.
Episode 1 policy loss: 24.644166946411133
Episode 1 total reward: -15377913811.52482
Episode 2 started.
Episode 2 finished after 1000 steps.
Episode 2 policy loss: 5.0995988845825195
Episode 2 total reward: -15377913718.22462
Episode 3 started.
Episode 3 finished after 1000 steps.
Episode 3 policy loss: 0.11301413178443909
Episode 3 total reward: -15377913718.124622
Episode 4 started.
Episode 4 finished after 1000 steps.
Episode 4 policy loss: 0.03307319059967995
Episode 4 total reward: -15377913718.124622
Episode 5 started.
Episode 5 finished after 1000 steps.
Episode 5 policy loss: 0.012178633362054825
Episode 5 total reward: -15377913718.124622
Episode 6 started.
Episode 6 finished after 1000 steps.
Episode 6 policy loss: 0.005270237103104591
Episode 6 total reward: -15377913718.124622
Episode 7 started.
Episode 7 finished after 1000 steps.
Episode 7 policy loss: 0.0025706554297357798
Episode 7 total reward: -15377913718.124622

In [2]:
# Actor Critic Method (Policy Based Method)

In [8]:
class ActorCriticNetwork(nn.Module):
    def __init__(self, input_size, output_size):
        super(ActorCriticNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 128)
        
        # Actor's layer
        self.action_head = nn.Linear(128, output_size)
        
        # Critic's layer
        self.value_head = nn.Linear(128, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        action_probs = torch.softmax(self.action_head(x), dim=-1)
        state_value = self.value_head(x)
        return action_probs, state_value

policy_net = ActorCriticNetwork(env.observation_space.shape[0], env.action_space.n)
optimizer = optim.Adam(policy_net.parameters(), lr=0.01)
gamma = 0.99

def select_action(state):
    state = torch.from_numpy(state).float().unsqueeze(0)
    probs, _ = policy_net(state)
    m = torch.distributions.Categorical(probs)
    action = m.sample()
    return action.item(), m.log_prob(action)

for episode in range(1000):
    state = env.reset()
    log_probs = []
    values = []
    rewards = []
    print(f"Episode {episode + 1} started.")
    
    for t in range(1000):
        action, log_prob = select_action(state)
        _, state_value = policy_net(torch.from_numpy(state).float().unsqueeze(0))
        
        next_state, reward, done, _ = env.step(action)
        rewards.append(reward)
        log_probs.append(log_prob)
        values.append(state_value)
        
        state = next_state
        if done:
            break
    
    print(f"Episode {episode + 1} finished after {t + 1} steps.")
    
    # Compute returns and advantages
    R = 0
    policy_loss = []
    value_loss = []
    returns = []
    for r in rewards[::-1]:
        R = r + gamma * R
        returns.insert(0, R)
    
    returns = torch.tensor(returns, dtype=torch.float32)
    returns = (returns - returns.mean()) / (returns.std() + 1e-9)
    
    for log_prob, value, R in zip(log_probs, values, returns):
        advantage = R - value.item()
        policy_loss.append(-log_prob * advantage)
        value_loss.append(nn.functional.mse_loss(value, torch.tensor([[R]])))

    optimizer.zero_grad()
    policy_loss = torch.stack(policy_loss).sum()
    value_loss = torch.stack(value_loss).sum()
    loss = policy_loss + value_loss
    loss.backward()
    optimizer.step()
    
    print(f"Episode {episode + 1} policy loss: {policy_loss.item()}")
    print(f"Episode {episode + 1} value loss: {value_loss.item()}")
    print(f"Episode {episode + 1} total reward: {sum(rewards)}")

print("Training completed.")

Episode 1 started.
Episode 1 finished after 1000 steps.
Episode 1 policy loss: 247.15013122558594
Episode 1 value loss: 4413455780020224.0
Episode 1 total reward: -15377913904.225014
Episode 2 started.
Episode 2 finished after 1000 steps.
Episode 2 policy loss: -1843.7442626953125
Episode 2 value loss: 3.6440955824583475e+17
Episode 2 total reward: -15377913904.92501
Episode 3 started.
Episode 3 finished after 1000 steps.
Episode 3 policy loss: -595.0410766601562
Episode 3 value loss: 4.591240153032294e+16
Episode 3 total reward: -15377913812.02482
Episode 4 started.
Episode 4 finished after 1000 steps.
Episode 4 policy loss: 340.16473388671875
Episode 4 value loss: 5647372503744512.0
Episode 4 total reward: -15377913806.324797
Episode 5 started.
Episode 5 finished after 1000 steps.
Episode 5 policy loss: 2957.985595703125
Episode 5 value loss: 3.0044677067505664e+16
Episode 5 total reward: -15377913803.724775
Episode 6 started.
Episode 6 finished after 1000 steps.
Episode 6 policy los

In [12]:
# Deep Q-Networks Algorithm (Deep Reinforcement Learning)

In [11]:
class DQN(nn.Module):
    def __init__(self, input_size, output_size):
        super(DQN, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, output_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

policy_net = DQN(env.observation_space.shape[0], env.action_space.n)
target_net = DQN(env.observation_space.shape[0], env.action_space.n)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

optimizer = optim.Adam(policy_net.parameters(), lr=0.001)
criterion = nn.MSELoss()

# Hyperparameters
gamma = 0.99
epsilon_start = 1.0
epsilon_end = 0.1
epsilon_decay = 500
target_update = 10
memory_capacity = 10000
batch_size = 64

memory = deque(maxlen=memory_capacity)

def select_action(state, epsilon):
    if random.random() > epsilon:
        with torch.no_grad():
            return policy_net(torch.from_numpy(state).float().unsqueeze(0)).argmax().item()
    else:
        return random.randrange(env.action_space.n)

def optimize_model():
    if len(memory) < batch_size:
        return

    batch = random.sample(memory, batch_size)
    states, actions, rewards, next_states, dones = zip(*batch)

    states = torch.tensor(np.array(states), dtype=torch.float32)
    actions = torch.tensor(actions)
    rewards = torch.tensor(rewards, dtype=torch.float32)
    next_states = torch.tensor(np.array(next_states), dtype=torch.float32)
    dones = torch.tensor(dones, dtype=torch.float32)

    current_q_values = policy_net(states).gather(1, actions.unsqueeze(1)).squeeze(1)
    max_next_q_values = target_net(next_states).max(1)[0]
    expected_q_values = rewards + (gamma * max_next_q_values * (1 - dones))

    loss = criterion(current_q_values, expected_q_values)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

for episode in range(1000):
    state = env.reset()
    total_reward = 0
    epsilon = epsilon_end + (epsilon_start - epsilon_end) * \
        np.exp(-1. * episode / epsilon_decay)
    for t in range(1000):
        action = select_action(state, epsilon)
        next_state, reward, done, _ = env.step(action)
        memory.append((state, action, reward, next_state, done))
        state = next_state
        total_reward += reward
        optimize_model()
        if done:
            break
    if episode % target_update == 0:
        target_net.load_state_dict(policy_net.state_dict())
    print(f"Episode {episode + 1} finished with total reward {total_reward}")

print("Training completed.")

Episode 1 finished with total reward -15377913804.924795
Episode 2 finished with total reward -15377913804.324799
Episode 3 finished with total reward -15377913807.024805
Episode 4 finished with total reward -15377913803.824808
Episode 5 finished with total reward -15377913804.824791
Episode 6 finished with total reward -15377913805.024797
Episode 7 finished with total reward -15377913807.524803
Episode 8 finished with total reward -15377913803.72478
Episode 9 finished with total reward -15377913805.824797
Episode 10 finished with total reward -15377913806.424788
Episode 11 finished with total reward -15377913800.424788
Episode 12 finished with total reward -15377913808.024797
Episode 13 finished with total reward -15377913805.024792
Episode 14 finished with total reward -15377913803.224787
Episode 15 finished with total reward -15377913800.52478
Episode 16 finished with total reward -15377913799.824795
Episode 17 finished with total reward -15377913803.124779
Episode 18 finished with 

In [13]:
# Double DQN Algorithm (Deep Reinforcement Learning)

In [14]:
class DQN(nn.Module):
    def __init__(self, input_size, output_size):
        super(DQN, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, output_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

policy_net = DQN(env.observation_space.shape[0], env.action_space.n)
target_net = DQN(env.observation_space.shape[0], env.action_space.n)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

optimizer = optim.Adam(policy_net.parameters(), lr=0.001)
criterion = nn.MSELoss()

# Hyperparameters
gamma = 0.99
epsilon_start = 1.0
epsilon_end = 0.1
epsilon_decay = 500
target_update = 10
memory_capacity = 10000
batch_size = 64

memory = deque(maxlen=memory_capacity)

def select_action(state, epsilon):
    if random.random() > epsilon:
        with torch.no_grad():
            return policy_net(torch.from_numpy(state).float().unsqueeze(0)).argmax().item()
    else:
        return random.randrange(env.action_space.n)

def optimize_model():
    if len(memory) < batch_size:
        return

    batch = random.sample(memory, batch_size)
    states, actions, rewards, next_states, dones = zip(*batch)

    states = torch.tensor(np.array(states), dtype=torch.float32)
    actions = torch.tensor(actions)
    rewards = torch.tensor(rewards, dtype=torch.float32)
    next_states = torch.tensor(np.array(next_states), dtype=torch.float32)
    dones = torch.tensor(dones, dtype=torch.float32)

    # Double DQN: Use policy_net to select the best action in next_state
    # Use target_net to evaluate this action
    current_q_values = policy_net(states).gather(1, actions.unsqueeze(1)).squeeze(1)
    next_actions = policy_net(next_states).argmax(1)
    next_q_values = target_net(next_states).gather(1, next_actions.unsqueeze(1)).squeeze(1)
    expected_q_values = rewards + (gamma * next_q_values * (1 - dones))

    loss = criterion(current_q_values, expected_q_values)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

for episode in range(1000):
    state = env.reset()
    total_reward = 0
    epsilon = epsilon_end + (epsilon_start - epsilon_end) * \
        np.exp(-1. * episode / epsilon_decay)
    for t in range(1000):
        action = select_action(state, epsilon)
        next_state, reward, done, _ = env.step(action)
        memory.append((state, action, reward, next_state, done))
        state = next_state
        total_reward += reward
        optimize_model()
        if done:
            break
    if episode % target_update == 0:
        target_net.load_state_dict(policy_net.state_dict())
    print(f"Episode {episode + 1} finished with total reward {total_reward}")

print("Training completed.")

Episode 1 finished with total reward -15377913808.424803
Episode 2 finished with total reward -15377913806.824804
Episode 3 finished with total reward -15377913805.924786
Episode 4 finished with total reward -15377913808.12481
Episode 5 finished with total reward -15377913801.624796
Episode 6 finished with total reward -15377913807.824806
Episode 7 finished with total reward -15377913805.624804
Episode 8 finished with total reward -15377913804.824804
Episode 9 finished with total reward -15377913801.924774
Episode 10 finished with total reward -15377913802.924797
Episode 11 finished with total reward -15377913804.424776
Episode 12 finished with total reward -15377913802.824785
Episode 13 finished with total reward -15377913805.124792
Episode 14 finished with total reward -15377913805.724773
Episode 15 finished with total reward -15377913806.524809
Episode 16 finished with total reward -15377913807.524807
Episode 17 finished with total reward -15377913807.924791
Episode 18 finished with

In [1]:
# Multi-Agent Reinforcement Learning Algorithm (MARL)

In [7]:
class QLearningAgent:
    def __init__(self, state_size, action_size, alpha=0.1, gamma=0.99, epsilon=0.1):
        self.state_size = state_size
        self.action_size = action_size
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.q_table = np.zeros((self.state_size, self.action_size))

    def select_action(self, state):
        if np.random.rand() < self.epsilon:
            return np.random.choice(self.action_size)
        else:
            return np.argmax(self.q_table[state])

    def update_q_table(self, state, action, reward, next_state):
        old_value = self.q_table[state, action]
        next_max = np.max(self.q_table[next_state])
        new_value = old_value + self.alpha * (reward + self.gamma * next_max - old_value)
        self.q_table[state, action] = new_value

# Initialize agents
num_agents = 2
agents = [QLearningAgent(env.observation_space.shape[0], env.action_space.n) for _ in range(num_agents)]

# Hyperparameters
num_episodes = 1000

# Training loop
for episode in range(num_episodes):
    state = env.reset()
    total_reward = 0
    done = False
    while not done:
        actions = [agent.select_action(0) for agent in agents]  # Assuming all agents start at the same state (0)
        next_state, reward, done, _ = env.step(actions)
        total_reward += reward  # Accumulate total reward
        
        # Update each agent's Q-table
        for agent, action in zip(agents, actions):
            agent.update_q_table(0, action, reward, 0)  # Hardcoding state index as 0 for simplicity
    
    print(f"Episode {episode + 1} completed with total reward: {total_reward}")

print("Training completed.")

Episode 1 completed with total reward: -15377913892.924969
Episode 2 completed with total reward: -15377913892.924969
Episode 3 completed with total reward: -15377913892.924969
Episode 4 completed with total reward: -15377913892.924969
Episode 5 completed with total reward: -15377913892.924969
Episode 6 completed with total reward: -15377913892.924969
Episode 7 completed with total reward: -15377913892.924969
Episode 8 completed with total reward: -15377913892.924969
Episode 9 completed with total reward: -15377913892.924969
Episode 10 completed with total reward: -15377913892.924969
Episode 11 completed with total reward: -15377913892.924969
Episode 12 completed with total reward: -15377913892.924969
Episode 13 completed with total reward: -15377913892.924969
Episode 14 completed with total reward: -15377913892.924969
Episode 15 completed with total reward: -15377913892.924969
Episode 16 completed with total reward: -15377913892.924969
Episode 17 completed with total reward: -15377913

In [1]:
# Options Framework (Hierarchical Reinforcement Learning)

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import gym
from gym import spaces
import numpy as np
import pandas as pd
import random
from collections import deque

class CentralBankingEnv(gym.Env):
    def __init__(self, data, target_inflation):
        super(CentralBankingEnv, self).__init__()
        self.data = data
        self.target_inflation = target_inflation
        self.current_step = 0

        # Define observation space based on selected columns (excluding non-numeric and irrelevant columns)
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(1,), dtype=np.float32)
        self.action_space = spaces.Discrete(3)  # Example: 0 = decrease rate, 1 = maintain, 2 = increase rate

    def step(self, action):
        reward = self.calculate_reward(action)
        self.current_step += 1
        if self.current_step >= len(self.data):
            done = True
            next_state = np.array([np.nan])
        else:
            next_state = np.array([self.data.iloc[self.current_step]['OBS_VALUE']])
            done = np.isnan(next_state).any()  # Check for NaN values
        return next_state, reward, done, {}

    def reset(self):
        self.current_step = 0
        return np.array([self.data.iloc[self.current_step]['OBS_VALUE']])

    def calculate_reward(self, action):
        current_inflation = self.data.iloc[self.current_step]['OBS_VALUE']
        if action == 0:  # decrease rate
            adjusted_inflation = current_inflation - 0.1
        elif action == 1:  # maintain rate
            adjusted_inflation = current_inflation
        else:  # increase rate
            adjusted_inflation = current_inflation + 0.1
        reward = -abs(adjusted_inflation - self.target_inflation)
        return reward

    def render(self, mode='human', close=False):
        print(f'Step: {self.current_step}, Inflation: {self.data.iloc[self.current_step]["OBS_VALUE"]}')

# Load the dataset
file_path = r'C:\Users\KIIT\Dropbox\Documents\Internship\ReBit\Daily Reports\Implementing_Methods\Main-aggregates-national-accounts-(International-Data-Cooperation).csv'
data = pd.read_csv(file_path)

# Initialize the environment with a target inflation rate
target_inflation = 2.0
env = CentralBankingEnv(data, target_inflation)

# Define the option and main policy networks
class OptionPolicy(nn.Module):
    def __init__(self, input_size, output_size):
        super(OptionPolicy, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, output_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.softmax(self.fc2(x), dim=-1)
        return x

class MainPolicy(nn.Module):
    def __init__(self, input_size, num_options):
        super(MainPolicy, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, num_options)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.softmax(self.fc2(x), dim=-1)
        return x

# Initialize policy networks
main_policy = MainPolicy(env.observation_space.shape[0], 2)  # Assuming 2 options
option_policies = [OptionPolicy(env.observation_space.shape[0], env.action_space.n) for _ in range(2)]

# Initialize optimizers
main_optimizer = optim.Adam(main_policy.parameters(), lr=0.001)
option_optimizers = [optim.Adam(option.parameters(), lr=0.001) for option in option_policies]

# Hyperparameters
gamma = 0.99
epsilon_start = 1.0
epsilon_end = 0.1
epsilon_decay = 500
memory_capacity = 10000
batch_size = 64
target_update = 10

memory = deque(maxlen=memory_capacity)

# Epsilon-greedy option selection
def select_option(state, epsilon):
    if random.random() > epsilon:
        with torch.no_grad():
            return main_policy(torch.from_numpy(state).float().unsqueeze(0)).argmax().item()
    else:
        return random.randrange(2)  # Number of options

# Epsilon-greedy action selection within an option
def select_action(option, state, epsilon):
    if random.random() > epsilon:
        with torch.no_grad():
            return option_policies[option](torch.from_numpy(state).float().unsqueeze(0)).argmax().item()
    else:
        return random.randrange(env.action_space.n)

def optimize_model():
    if len(memory) < batch_size:
        return

    batch = random.sample(memory, batch_size)
    states, options, actions, rewards, next_states, dones = zip(*batch)

    states = torch.tensor(np.array(states), dtype=torch.float32)
    options = torch.tensor(options)
    actions = torch.tensor(actions)
    rewards = torch.tensor(rewards, dtype=torch.float32)
    next_states = torch.tensor(np.array(next_states), dtype=torch.float32)
    dones = torch.tensor(dones, dtype=torch.float32)

    # Main policy optimization
    current_option_q_values = main_policy(states).gather(1, options.unsqueeze(1)).squeeze(1)
    next_option_q_values = main_policy(next_states).max(1)[0]
    expected_option_q_values = rewards + (gamma * next_option_q_values * (1 - dones))

    option_loss = nn.functional.mse_loss(current_option_q_values, expected_option_q_values)
    main_optimizer.zero_grad()
    option_loss.backward()
    main_optimizer.step()

    # Option policies optimization
    for option in range(2):
        mask = options == option
        if mask.sum() == 0:
            continue

        option_states = states[mask]
        option_actions = actions[mask]
        option_rewards = rewards[mask]
        option_next_states = next_states[mask]
        option_dones = dones[mask]

        current_q_values = option_policies[option](option_states).gather(1, option_actions.unsqueeze(1)).squeeze(1)
        next_q_values = option_policies[option](option_next_states).max(1)[0]
        expected_q_values = option_rewards + (gamma * next_q_values * (1 - option_dones))

        loss = nn.functional.mse_loss(current_q_values, expected_q_values)
        option_optimizers[option].zero_grad()
        loss.backward()
        option_optimizers[option].step()

for episode in range(1000):
    state = env.reset()
    total_reward = 0
    epsilon = epsilon_end + (epsilon_start - epsilon_end) * np.exp(-1. * episode / epsilon_decay)
    
    while True:
        option = select_option(state, epsilon)
        action = select_action(option, state, epsilon)
        
        next_state, reward, done, _ = env.step(action)
        memory.append((state, option, action, reward, next_state, done))
        
        state = next_state
        total_reward += reward
        
        optimize_model()
        
        if done:
            break
    
    if episode % target_update == 0:
        for option in range(2):
            target_net = OptionPolicy(env.observation_space.shape[0], env.action_space.n)
            target_net.load_state_dict(option_policies[option].state_dict())
    
    print(f"Episode {episode + 1} finished with total reward {total_reward}")

print("Training completed.")


Episode 1 finished with total reward -15377913801.224796
Episode 2 finished with total reward -15377913808.52481
Episode 3 finished with total reward -15377913809.524818
Episode 4 finished with total reward -15377913808.424803
Episode 5 finished with total reward -15377913805.42478
Episode 6 finished with total reward -15377913801.224789
Episode 7 finished with total reward -15377913807.624802
Episode 8 finished with total reward -15377913805.224794
Episode 9 finished with total reward -15377913808.324806
Episode 10 finished with total reward -15377913807.424799
Episode 11 finished with total reward -15377913805.524782
Episode 12 finished with total reward -15377913801.92478
Episode 13 finished with total reward -15377913803.224815
Episode 14 finished with total reward -15377913804.32478
Episode 15 finished with total reward -15377913797.424782
Episode 16 finished with total reward -15377913805.324814
Episode 17 finished with total reward -15377913801.924772
Episode 18 finished with to

In [2]:
# Feudal Reinforcement Learning (Hierarchical Reinforcement Learning)

In [14]:
from collections import deque

# Define Manager and Worker networks
class ManagerNetwork(nn.Module):
    def __init__(self, input_size, goal_size):
        super(ManagerNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, goal_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        goal = torch.tanh(self.fc2(x))  
        return goal

class WorkerNetwork(nn.Module):
    def __init__(self, input_size, goal_size, action_size):
        super(WorkerNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size + goal_size, 128)
        self.fc2 = nn.Linear(128, action_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        action_probs = torch.softmax(self.fc2(x), dim=-1)
        return action_probs

# Initialize networks and optimizers
goal_size = 1
manager = ManagerNetwork(env.observation_space.shape[0], goal_size)
worker = WorkerNetwork(env.observation_space.shape[0], goal_size, env.action_space.n)

manager_optimizer = optim.Adam(manager.parameters(), lr=0.001)
worker_optimizer = optim.Adam(worker.parameters(), lr=0.001)

# Hyperparameters
gamma = 0.99
goal_update_frequency = 5
memory_capacity = 10000
batch_size = 64

memory = deque(maxlen=memory_capacity)

def select_goal(state):
    with torch.no_grad():
        goal = manager(torch.from_numpy(state).float().unsqueeze(0)).squeeze(0)
    return goal

def select_action(state, goal):
    state = torch.from_numpy(state).float()
    state_goal = torch.cat((state, goal.float()), dim=-1)  
    
    with torch.no_grad():
        action_probs = worker(state_goal.unsqueeze(0)).squeeze(0)
    
    m = torch.distributions.Categorical(action_probs)
    action = m.sample()
    return action.item(), m.log_prob(action)

def compute_intrinsic_rewards(state_goal):
    return torch.zeros(state_goal.size(0))

def optimize_model():
    if len(memory) < batch_size:
        return
    
    batch = random.sample(memory, batch_size)
    states, goals, actions, rewards, next_states, dones = zip(*batch)
    
    states = torch.tensor(np.array(states), dtype=torch.float32)
    goals = torch.tensor(np.array(goals), dtype=torch.float32)  
    goals = goals.unsqueeze(1) if len(goals.shape) < 2 else goals  
    
    actions = torch.tensor(actions)
    rewards = torch.tensor(rewards, dtype=torch.float32)
    next_states = torch.tensor(np.array(next_states), dtype=torch.float32)
    dones = torch.tensor(dones, dtype=torch.float32)
    
    state_goal = torch.cat((states, goals), dim=1)
    
    action_probs = worker(state_goal).gather(1, actions.unsqueeze(1)).squeeze(1)
    
    # Compute intrinsic_rewards
    intrinsic_rewards = compute_intrinsic_rewards(state_goal)
    
    expected_rewards = rewards + gamma * (1 - dones) * intrinsic_rewards
    
    manager_optimizer.zero_grad()
    worker_optimizer.zero_grad()
    
    loss = -torch.mean(action_probs * expected_rewards)
    loss.backward()
    
    manager_optimizer.step()
    worker_optimizer.step()

for episode in range(1000):
    state = env.reset()
    total_reward = 0
    goal = select_goal(state)
    
    for t in range(1000):
        if t % goal_update_frequency == 0:
            goal = select_goal(state)
        
        action, log_prob = select_action(state, goal)
        next_state, reward, done, _ = env.step(action)
        
        memory.append((state, goal.numpy(), action, reward, next_state, done))
        
        state = next_state
        total_reward += reward
        
        optimize_model()
        
        if done:
            break
    
    print(f"Episode {episode + 1} finished with total reward {total_reward}")

print("Training completed.")


Episode 1 finished with total reward -15377913809.52477
Episode 2 finished with total reward -15377913892.924969
Episode 3 finished with total reward -15377913892.924969
Episode 4 finished with total reward -15377913892.924969
Episode 5 finished with total reward -15377913892.924969
Episode 6 finished with total reward -15377913783.524672
Episode 7 finished with total reward -15377913705.524578
Episode 8 finished with total reward -15377913705.524578
Episode 9 finished with total reward -15377913705.524578
Episode 10 finished with total reward -15377913705.524578
Episode 11 finished with total reward -15377913705.524578
Episode 12 finished with total reward -15377913705.524578
Episode 13 finished with total reward -15377913705.524578
Episode 14 finished with total reward -15377913859.724993
Episode 15 finished with total reward -15377913892.924969
Episode 16 finished with total reward -15377913892.924969
Episode 17 finished with total reward -15377913892.924969
Episode 18 finished with